In [2]:
import os
import time
import json

import concurrent
import numpy as np
import pandas as pd
from tqdm import tqdm
from evaluation_functions import evaluate
from openai_functions import extract_ade_terms
from sentence_transformers import SentenceTransformer

## Set Up

### Variables

In [7]:
# train
# drug_file = 'data/train_drug_label_text.csv'
# manual_file = 'data/train_drug_label_text_manual_ades.csv'

# test
drug_file = 'data/TAC2017/test_drug_label_text.csv'
manual_file = 'data/TAC2017/test_drug_label_text_manual_ades.csv'

# my_max = 10000

In [19]:
drugs = pd.read_csv(drug_file)
manual_ades = pd.read_csv(manual_file)
set_type = drug_file.split('/')[2].split('_')[0] # assuming file follows format "train_..." or "test...."

all_sections = drugs.query("section_name != 'all-concat'").groupby('drug_name')['section_text'].apply(' '.join).reset_index()
all_sections.insert(1, "section_name", ["all-concat" for _ in range(all_sections.shape[0])])
drugs = pd.concat([drugs, all_sections])

set_type

'test'

In [20]:
drugs.shape

(336, 3)

## Run GPT

In [21]:
outputs = {}

In [22]:
config = json.load(open('./config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint'] 

gpt_model = config[api_source]["gpt_model"]
gpt_model = "gpt-4-turbo-preview"
# gpt_model = "gpt-3.5-turbo-0125"

temperature = 0

In [23]:
nruns = 1

system_options = {
    "no-system-prompt": "",
    "pharmexpert-v0": "You are an expert in pharmacology.",
    "pharmexpert-v1": "You are an expert in medical natural language processing, adverse drug reactions, pharmacology, and clinical trials."
}

prompt_options = {
    "fatal-prompt-v1": """
Extract all adverse reactions and fatal outcomes as they appear, including all synonyms,
mentioned in the following text provide them as a comma-separated list:
'{}'
""",
    "fatal-prompt-v2": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list.
If a fatal event is listed add 'death' to the list.
The text is :'{}' 
""",
    "fatal-prompt-v3": """
Extract all adverse reactions as they appear, including all synonyms.
mentioned in the text and provide them as a comma-separated list. If a
negated adverse reaction appears extract it and include a <negated> tag.
If a fatal event is listed add 'death' to the list.
The text is :'{}'
""",
    "only-positives-v0": """
Extract all adverse events as they appear, including all synonyms mentioned in the text
and provide them as a comma separated list. Only include adverse events that
have evidence of a causal relationship to the drug exposure. If a fatal event
is listed add 'death' to to the list.
The text is :'{}'
""",
    "gpt-written-prompt":"""
Given the structured product label below, extract information on adverse drug reactions and provide the exact mentions of reactions
in a comma-separated format. Consider sentence-form expressions, reactions in tables, negated reactions, discontinuous mentions,
hypothetical scenarios, and potentially fatal occurrences. Ensure the extraction is comprehensive, covering both explicit and 
implicit references to adverse reactions.
The text from the structured product label is:
'{}'
"""
}

system_name = "pharmexpert-v0"
system_content = system_options[system_name]

prompt_name = "only-positives-v0"
prompt = prompt_options[prompt_name]

gpt_params = [f"temp{temperature}"]

output_file_basename = '{}_{}_{}_{}_{}_{}'.format(api_source, gpt_model, prompt_name, system_name, '-'.join(gpt_params), set_type)
output_file_basename

'OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test'

In [25]:
# run GPT
for i in range(nruns):
    run_key = "{}_run{}".format(output_file_basename, i)
    print(run_key)
    if run_key in outputs:
        print(f"Run {run_key} already started will pick up from where it was left off.")
    elif os.path.exists('results/{}.csv'.format(run_key)):
        gpt_output = pd.read_csv('results/{}.csv'.format(run_key))
        outputs[run_key] = gpt_output
        print(f"Run {run_key} started, loading from disk and pick up from where it was left off.")
    
    start = time.time()
    results = list()
    rows_to_run = list()
    for _, row in drugs.iterrows():
        name, section = row['drug_name'], row['section_name']

        if run_key in outputs:
            prev_run_results = outputs[run_key].query(f"drug_name == '{name}'").query(f"section_name == '{section}'")
            if prev_run_results.shape[0]==1:
                results.append([name, section, prev_run_results.gpt_output.values[0]])
            else:
                rows_to_run.append(row)
        else:
            rows_to_run.append(row)
        
    print(f"Loaded {len(results)} rows from file since they were already run.")
    print(f"There remains {len(rows_to_run)} rows to run.")

    def run_iteration(row):
        name, section = row['drug_name'], row['section_name']
        text = row['section_text']
        try:
            gpt_out = extract_ade_terms(api_source, config, gpt_model, system_content, prompt, text, temperature)
            return [name, section, gpt_out]
        except Exception as err:
            print(f"Encountered an exception for row: {name} {section}. Error message below:")
            print(err)
            return None
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as exec:
        results.extend(list(tqdm(
            exec.map(run_iteration, rows_to_run), 
            total=len(rows_to_run)
        )))
            
    gpt_output = pd.DataFrame(
        [r for r in results if r is not None],
        columns=['drug_name', 'section_name', 'gpt_output']
    )
    end = time.time()
    
    if gpt_output.shape[0] > 0:
        outputs[run_key] = gpt_output
        gpt_output.to_csv('results/{}.csv'.format(run_key))
    
    print(f"Run: {run_key}, time elapsed: {end-start}s.")

OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0
Loaded 0 rows from file since they were already run.
There remains 340 rows to run.


100%|█████████████████████████████████████████| 340/340 [08:13<00:00,  1.45s/it]


Run: OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0, time elapsed: 493.6048946380615s.
OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1
Loaded 0 rows from file since they were already run.
There remains 340 rows to run.


100%|█████████████████████████████████████████| 340/340 [07:57<00:00,  1.41s/it]


Run: OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1, time elapsed: 478.0281710624695s.
OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2
Loaded 0 rows from file since they were already run.
There remains 340 rows to run.


100%|█████████████████████████████████████████| 340/340 [07:55<00:00,  1.40s/it]

Run: OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0, time elapsed: 0.2586100101470947s.


## Evaluation

In [26]:
for run_key in sorted(outputs.keys()):
    print(run_key)

OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0


In [27]:
# gpt = {'thrombocytopenia', 'anemia', 'death', 'acute renal failure', 'pulmonary toxicity', 'hepatic failure', 'nausea', 'infusion reactions', 'pulmonary hypertension', 'hepatic toxicity', 'decreased lymphocyte', 'dyspnea', 'tumor lysis syndrome', 'diarrhea', 'fatigue', 'pyrexia', 'edema peripheral', 'hypertension', 'decreased platelets', 'cardiac toxicities', 'hemolytic uremic syndrome', 'headache', 'venous thrombosis', 'thrombotic thrombocytopenic purpura', 'cough', 'posterior reversible encephalopathy syndrome'}
# man = {'cardiac failure congestive', 'died', 'thrombocytopenia', 'decreased total white blood cell count', 'leukopenia', 'dizziness', 'deafness', 'pulmonary toxicity', 'hepatic failure', 'pain in extremity', 'asthenia', 'cataract', 'rash', 'bronchopneumonia', 'hypophosphatemia', 'hypoalbuminemia', 'bronchitis', 'tumor lysis syndrome', 'diarrhea', 'decreased absolute neutrophil count', 'pyrexia', 'edema peripheral', 'musculoskeletal pain', 'muscle spasm', 'hypertension', 'ttp', 'cardiac toxicities', 'decreased phosphorus', 'myocardial ischemia', 'infection adverse events', 'anxiety', 'hypomagnesemia', 'decreased potassium', 'posterior reversible encephalopathy syndrome', 'insomnia', 'mortality', 'decreased appetite', 'hypoesthesia', 'dysphonia', 'viral infection', 'renal failure', 'hypercalcemia', 'febrile neutropenia', 'pain', 'lymphopenia', 'renal disorders', 'respiratory tract infection', 'infusion reactions', 'disease progression', 'congestive heart failure', 'pruritus', 'pneumonia', 'upper respiratory tract infection', 'hepatic toxicity', 'renal impairment', 'chills', 'muscle spasms', 'back pain', 'deaths', 'epistaxis', 'decreased sodium', 'hemolytic uremic syndrome', 'oropharyngeal pain', 'thrombotic thrombocytopenic purpura', 'dehydration', 'hyperkalemia', 'musculoskeletal chest pain', 'hypocalcemia', 'hyperuricemia', 'myalgia', 'dyspnea exertional', 'peripheral neuropathies nec', 'blurred vision', 'erythema', 'delirium', 'acute renal failure', 'renal failure acute', 'nausea', 'vomiting', 'nasopharyngitis', 'sepsis', 'hypotension', 'decreased lymphocyte', 'fatigue', 'fatal', 'hypertensive crisis', 'vision blurred', 'deep vein thrombosis', 'infusion site reaction', 'embolic and thrombotic events, venous', 'headache', 'cardiac failure', 'decreased lymphocytes', 'cardiac adverse events', 'hypertensive emergency', 'pulmonary edema', 'pulmonary embolism', 'myocardial infarction', 'decreased hemoglobin', 'influenza', 'paresthesia', 'hyperglycemia', 'neutropenia', 'abdominal pain upper', 'anemia', 'toothache', 'cardiac disorders', 'multi-organ failure', 'cardiac arrest', 'pres', 'hyperhidrosis', 'pulmonary hypertension', 'hypokalemia', 'dyspnea', 'dyspepsia', 'urinary tract infection', 'abdominal pain', 'decreased platelets', 'renal adverse events', 'venous thrombosis', 'muscular weakness', 'infections', 'cough', 'hus', 'arthralgia', 'multiple myeloma', 'constipation', 'hyponatremia'}
# len(man)

In [28]:
evaluate(outputs, manual_ades, 'strict')
evaluate(outputs, manual_ades, 'lenient')

/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  output['gpt_output'] = output['gpt_output'].str.lower().str.replace('.', '').str.replace('\n-', ', ').str.split(', ')


Running strict evaluation and saving results to disk.
OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0
saving results to results/evals/OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0_strict_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0_strict_overall.csv


100%|██████████| 99/99 [00:01<00:00, 59.47it/s]
/Users/TatonettiN/Projects/onsides-task1/evaluation_functions.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  output['gpt_output'] = output['gpt_output'].str.lower().str.replace('.', '').str.replace('\n-', ', ').str.split(', ')


Running lenient evaluation and saving results to disk.
OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0
saving results to results/evals/OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0_lenient_granular.csv and results/evals/OpenAI_gpt-4-1106-preview_only-positives-v0_pharmexpert-v0_temp0_test_run0_lenient_overall.csv


100%|█████████████████████████████████████████| 101/101 [00:26<00:00,  3.82it/s]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_lenient_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_lenient_overall.csv


100%|█████████████████████████████████████████| 101/101 [00:25<00:00,  3.89it/s]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_lenient_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_lenient_overall.csv


100%|█████████████████████████████████████████| 101/101 [00:25<00:00,  3.90it/s]


In [12]:
# if using embeddings -- run this once:
# get embeddings for manual annotation --- this part is slow -- but should take <5 min
embed_model_name = 'llmrails/ember-v1'
embed_model = SentenceTransformer(embed_model_name)
man_embeds = embed_model.encode(manual_ades['reaction_string'].tolist())
manual_ades['embeds'] = list(man_embeds)

/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/uog2000/miniconda3/envs/llm_cpus/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [13]:
evaluate(outputs, manual_ades, 'embed', embed_model=embed_model, embed_model_name=embed_model_name)

Running embed evaluation and saving results to disk.
OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run0_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [05:32<00:00,  3.29s/it]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run1_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [05:20<00:00,  3.17s/it]


OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2
saving results to results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_ember-v1_granular.csv and results/OpenAI_gpt-4-turbo-preview_gpt-written-prompt_pharmexpert-v0_temp0_train_run2_ember-v1_overall.csv


100%|█████████████████████████████████████████| 101/101 [06:39<00:00,  3.96s/it]
